# Data Extraction and Preprocessing


## Demographics

### Extracting Demographic information

In [ ]:
import os
import pandas as pd

# Set base path to WESAD dataset folder
base_path = "/content/drive/MyDrive/Capstone/WESAD"
output_file = "/content/drive/MyDrive/Capstone/demographic_info.csv"

# Initialize a list to store patient demographic data
patient_data = []

# Function to process a readme file
def process_readme_file(readme_path):
    """
    Extract demographic information from the readme file.

    Parameters:
        readme_path (str): Path to the readme file.

    Returns:
        dict: A dictionary containing the extracted demographic data.
    """
    demographics = {}
    try:
        with open(readme_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                line = line.strip()
                # Skip empty lines or lines without ":"
                if not line or ":" not in line:
                    continue

                key, value = map(str.strip, line.split(":", 1))  # Split on the first ":"

                # Map keys to corresponding fields
                if key == "Age":
                    demographics["Age"] = int(value)
                elif key == "Height (cm)":
                    demographics["Height_cm"] = int(value)
                elif key == "Weight (kg)":
                    demographics["Weight_kg"] = int(value)
                elif key == "Gender":
                    demographics["Gender"] = value
                elif key == "Dominant hand":
                    demographics["Dominant_Hand"] = value
                elif key == "Did you drink coffee today?":
                    demographics["Coffee_Today"] = value
                elif key == "Did you drink coffee within the last hour?":
                    demographics["Coffee_Last_Hour"] = value
                elif key == "Did you do any sports today?":
                    demographics["Sports_Today"] = value
                elif key == "Are you a smoker?":
                    demographics["Smoker"] = value
                elif key == "Did you smoke within the last hour?":
                    demographics["Smoke_Last_Hour"] = value
                elif key == "Do you feel ill today?":
                    demographics["Feel_Ill"] = value

                # Handle additional notes
                elif key == "### Additional notes ###":
                    # Collect additional notes from the subsequent lines
                    notes_index = lines.index(line) + 1
                    additional_notes = " ".join([l.strip() for l in lines[notes_index:]])
                    demographics["Additional_Notes"] = additional_notes
                    break  # Stop processing further lines
    except Exception as e:
        print(f"Error processing {readme_path}: {e}")
    return demographics


# Process each patient's readme file
for patient_folder in [os.path.join(base_path, f) for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]:
    patient_id = os.path.basename(patient_folder)
    readme_path = os.path.join(patient_folder, f"{patient_id}_readme.txt")

    if os.path.exists(readme_path):
        print(f"Processing {readme_path}")
        demographics = process_readme_file(readme_path)
        demographics["Patient_ID"] = patient_id  # Add patient ID
        patient_data.append(demographics)
    else:
        print(f"Readme file not found for {patient_id}")

# Create a DataFrame from the collected data
demographics_df = pd.DataFrame(patient_data)

# Save DataFrame to CSV
demographics_df.to_csv(output_file, index=False)
print(f"Demographic information saved to {output_file}")


## Protocols

### Extract protocols

In [ ]:
import os
import pandas as pd

# Set paths
output_folder_quest = "/content/drive/MyDrive/Capstone/output_quest_data"  # Path to save questionnaire data
os.makedirs(output_folder_quest, exist_ok=True)

# Define required conditions
REQUIRED_CONDITIONS = {"Base", "TSST", "Medi 1", "Fun", "Medi 2"}

def extract_protocol_data(quest_path, patient_id):
    """
    Extracts study protocol conditions and start/end times from the patient's quest file.
    Filters only the required conditions: Base, TSST, Medi 1, Fun, and Medi 2.
    Saves the data as a CSV file.
    """
    try:
        # Read the questionnaire file
        df = pd.read_csv(quest_path, header=None, sep=";", skip_blank_lines=False)

        # Initialize processed data dictionary
        processed_data = {"Condition": [], "Start_Time": [], "End_Time": []}

        # Extract Conditions
        conditions_row = df[df[0].str.contains("ORDER", na=False)]
        if conditions_row.empty:
            print(f"Warning: Missing conditions row for {patient_id}")
            return

        conditions = conditions_row.iloc[0, 1:].dropna().tolist()

        # Extract START and END times
        start_times_row = df[df[0].str.contains("START", na=False)]
        end_times_row = df[df[0].str.contains("END", na=False)]

        if start_times_row.empty or end_times_row.empty:
            print(f"Warning: Missing start/end times for {patient_id}")
            return

        start_times = start_times_row.iloc[0, 1:].dropna().tolist()
        end_times = end_times_row.iloc[0, 1:].dropna().tolist()

        # Ensure alignment between conditions and times
        min_length = min(len(conditions), len(start_times), len(end_times))
        conditions = conditions[:min_length]
        start_times = start_times[:min_length]
        end_times = end_times[:min_length]

        # Filter only required conditions
        for cond, start, end in zip(conditions, start_times, end_times):
            if cond in REQUIRED_CONDITIONS:
                try:
                    if start.isdigit():
                        start_sec = float(start) * 60
                    elif "." in start:
                        start_sec = float(start.split(".")[0]) * 60 + float(start.split(".")[1])
                    else:
                        start_sec = None

                    if end.isdigit():
                        end_sec = float(end) * 60
                    elif "." in end:
                        end_sec = float(end.split(".")[0]) * 60 + float(end.split(".")[1])
                    else:
                        end_sec = None

                    processed_data["Condition"].append(cond)
                    processed_data["Start_Time"].append(start_sec)
                    processed_data["End_Time"].append(end_sec)
                except Exception as e:
                    print(f"Error converting times for {patient_id} in condition {cond}: {e}")

        # Create DataFrame and save
        protocol_df = pd.DataFrame(processed_data)
        output_file = os.path.join(output_folder_quest, f"{patient_id}_protocol.csv")
        protocol_df.to_csv(output_file, index=False)
        print(f"Protocol data saved for {patient_id} to {output_file}")

    except Exception as e:
        print(f"Error processing protocol data for {patient_id}: {e}")

# Function to process all patients
def process_protocols_all_patients(base_path):
    """
    Iterate through all patient folders and extract protocol data.
    """
    for patient_folder in [os.path.join(base_path, f) for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]:
        patient_id = os.path.basename(patient_folder)
        quest_path = os.path.join(patient_folder, f"{patient_id}_quest.csv")
        if os.path.exists(quest_path):
            extract_protocol_data(quest_path, patient_id)

# Example Usage:
# Set base path where patient folders are located
base_path = "/content/drive/MyDrive/Capstone/WESAD"

# Run the script for all patients
process_protocols_all_patients(base_path)


### Complied protocols

In [ ]:
import os

# Define the output folder where protocol files are stored
output_folder_quest = "/content/drive/MyDrive/Capstone/output_quest_data"

# List all protocol files
protocol_files = [f for f in os.listdir(output_folder_quest) if f.endswith("_protocol.csv")]

# Initialize an empty list to store data from all patients
all_data = []

# Loop through each file and compile data
for file in protocol_files:
    patient_id = file.split("_protocol.csv")[0]  # Extract patient ID from file name
    file_path = os.path.join(output_folder_quest, file)

    # Read the patient's protocol file
    df = pd.read_csv(file_path)

    # Add patient ID column
    df.insert(0, "Patient_ID", patient_id)

    # Append to the list
    all_data.append(df)

# Combine all patient data into a single DataFrame
compiled_df = pd.concat(all_data, ignore_index=True)

#Add column for test duration
compiled_df['Duration'] = compiled_df['End_Time'] - compiled_df['Start_Time']

# Define output path for the compiled CSV file
compiled_file_path = os.path.join(output_folder_quest, "compiled_protocol_data.csv")

# Save the combined data to a CSV file
compiled_df.to_csv(compiled_file_path, index=False)


## Questionnaire

### Extract questionnaire data

In [ ]:
import os
import pandas as pd

# Set paths
output_folder_quest = "/content/drive/MyDrive/Capstone/output_quest_data"  # Path to save questionnaire data
os.makedirs(output_folder_quest, exist_ok=True)

def extract_survey_data(quest_path, patient_id):
    """
    Extract PANAS, STAI, DIM, and SSSQ questionnaire responses from the patient's quest file.
    Saves each survey as a separate CSV file.
    """
    try:
        # Read the questionnaire file
        df = pd.read_csv(quest_path, header=None, sep=";", skip_blank_lines=False)

        # Define survey types and their corresponding columns
        survey_columns = {
            "PANAS": [
                "PANAS_Active", "PANAS_Distressed", "PANAS_Interested", "PANAS_Inspired",
                "PANAS_Annoyed", "PANAS_Strong", "PANAS_Guilty", "PANAS_Scared",
                "PANAS_Hostile", "PANAS_Excited", "PANAS_Proud", "PANAS_Irritable",
                "PANAS_Enthusiastic", "PANAS_Ashamed", "PANAS_Alert", "PANAS_Nervous",
                "PANAS_Determined", "PANAS_Attentive", "PANAS_Jittery", "PANAS_Afraid",
                "PANAS_Stressed", "PANAS_Frustrated", "PANAS_Happy", "PANAS_Angry",
                "PANAS_Irritated", "PANAS_Sad"
            ],
            "STAI": [
                "STAI_ease", "STAI_nervous", "STAI_jittery", "STAI_relaxed",
                "STAI_worried", "STAI_pleasant"
            ],
            "DIM": ["SAM_valence", "SAM_arousal"]
          }

        # Define conditions
        conditions = ["Base", "TSST", "Medi 1", "Fun", "Medi 2"]

        # Extract and save each survey separately
        for survey, columns in survey_columns.items():
            survey_rows = df[df[0].str.contains(survey, na=False)]
            if not survey_rows.empty:
                survey_data = survey_rows.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')  # Convert to numeric

                if survey == "PANAS":
                  survey_data.columns = columns[:survey_data.shape[1]]  # Assign columns based on survey type

                elif survey == "STAI":
                  survey_data = survey_data.iloc[:, :6]  # Keep only the first 6 columns
                  survey_data.columns = survey_columns["STAI"]

                elif survey == "DIM":
                  survey_data = survey_data.iloc[:, :2]  # Keep only the first 2 columns
                  survey_data.columns = survey_columns["DIM"]


                # Add CONDITION column
                survey_data["CONDITION"] = conditions[:len(survey_data)]

                # Special handling for PANAS_Angry and PANAS_Irritated (only filled in TSST condition)
                if survey == "PANAS":
                    survey_data.loc[survey_data["CONDITION"] != "TSST", ["PANAS_Sad"]] = survey_data['PANAS_Angry']
                    survey_data.loc[survey_data["CONDITION"] != "TSST", ["PANAS_Angry", "PANAS_Irritated"]] = float('NaN')

                # Save to CSV
                output_file = os.path.join(output_folder_quest, f"{patient_id}_{survey}.csv")
                survey_data.to_csv(output_file, index=False)
                print(f"Saved {survey} data for {patient_id} to {output_file}")

    except Exception as e:
        print(f"Error processing questionnaire data for {patient_id}: {e}")

# Function to process all patients
def process_survey_data_all_patients(base_path):
    """
    Iterate through all patient folders and extract questionnaire data.
    """
    for patient_folder in [os.path.join(base_path, f) for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]:
        patient_id = os.path.basename(patient_folder)
        quest_path = os.path.join(patient_folder, f"{patient_id}_quest.csv")
        if os.path.exists(quest_path):
            extract_survey_data(quest_path, patient_id)

# Example Usage:
# Set base path where patient folders are located
base_path = "/content/drive/MyDrive/Capstone/WESAD"

# Run the script for all patients
process_survey_data_all_patients(base_path)

### Combine questionnaire data for each patient

In [ ]:
import os
import pandas as pd

# Define the folder paths where the extracted survey data is saved
output_folder_quest = "/content/drive/MyDrive/Capstone/output_quest_data"
output_folder_merged = "/content/drive/MyDrive/Capstone/output_merged_quest_data"
os.makedirs(output_folder_merged, exist_ok=True)

def merge_surveys(patient_id):
    """
    Merge PANAS, STAI, and DIM CSV files into one CSV for each patient.
    """
    try:
        # Define paths to the individual survey and protocol files
        panas_file = os.path.join(output_folder_quest, f"{patient_id}_PANAS.csv")
        stai_file = os.path.join(output_folder_quest, f"{patient_id}_STAI.csv")
        dim_file = os.path.join(output_folder_quest, f"{patient_id}_DIM.csv")
        protocol_file = os.path.join(output_folder_quest, f"{patient_id}_protocol.csv")

        # Check if all files exist before proceeding
        if os.path.exists(panas_file) and os.path.exists(stai_file) and os.path.exists(dim_file):
            # Read the survey CSV files
            panas_df = pd.read_csv(panas_file)
            stai_df = pd.read_csv(stai_file)
            dim_df = pd.read_csv(dim_file)
            protocol_df = pd.read_csv(protocol_file)

            # Rename 'Condition' in protocol to match 'CONDITION' in other files
            protocol_df.rename(columns={'Condition': 'CONDITION'}, inplace=True)

            # Merge the dataframes on the 'CONDITION' column
            merged_df = panas_df.merge(stai_df, on='CONDITION').merge(dim_df, on='CONDITION').merge(protocol_df, on='CONDITION')

            # Reorder columns to ensure CONDITION, Start_Time, and End_Time are first
            cols = merged_df.columns.tolist()
            cols.insert(0, cols.pop(cols.index('CONDITION')))
            cols.insert(1, cols.pop(cols.index('Start_Time')))
            cols.insert(2, cols.pop(cols.index('End_Time')))
            merged_df = merged_df[cols]

            # Save the merged dataframe to a new CSV file
            merged_output_file = os.path.join(output_folder_merged, f"{patient_id}_merged.csv")
            merged_df.to_csv(merged_output_file, index=False)
            print(f"Merged survey data saved for {patient_id} to {merged_output_file}")
        else:
            print(f"Missing one or more survey files for {patient_id}. Skipping merge.")

    except Exception as e:
        print(f"Error merging survey data for {patient_id}: {e}")

# Function to process and merge data for all patients
def process_and_merge_all_patients(base_path):
    """
    Extract questionnaire data and merge PANAS, STAI, and DIM for all patients.
    """
    for patient_folder in [os.path.join(base_path, f) for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]:
        patient_id = os.path.basename(patient_folder)
        merge_surveys(patient_id)

# Example usage
base_path = "/content/drive/MyDrive/Capstone/WESAD"
process_and_merge_all_patients(base_path)


### Compile questionnaire data for all patients

In [ ]:
import os
import pandas as pd

# Define folder paths
output_folder_quest = "/content/drive/MyDrive/Capstone/output_quest_data"
protocol_file = "/content/drive/MyDrive/Capstone/output_quest_data/compiled_protocol_data.csv"
output_merged_file = "/content/drive/MyDrive/Capstone/compiled_survey_data.csv"

# Load protocol data
protocol_data = pd.read_csv(protocol_file)

# Initialize list to collect all merged survey data
all_survey_data = []

# Function to merge surveys for a single patient
def merge_surveys(patient_id, output_folder_quest, protocol_data):
    """
    Merge PANAS, STAI, and DIM survey data into a single dataset for a patient.
    """
    try:
        # Define paths to individual survey files
        panas_file = os.path.join(output_folder_quest, f"{patient_id}_PANAS.csv")
        stai_file = os.path.join(output_folder_quest, f"{patient_id}_STAI.csv")
        dim_file = os.path.join(output_folder_quest, f"{patient_id}_DIM.csv")

        # Check if all survey files exist before proceeding
        if os.path.exists(panas_file) and os.path.exists(stai_file) and os.path.exists(dim_file):
            # Read the survey CSV files
            panas_df = pd.read_csv(panas_file)
            stai_df = pd.read_csv(stai_file)
            dim_df = pd.read_csv(dim_file)

            # Merge survey data on CONDITION column
            merged_df = panas_df.merge(stai_df, on='CONDITION').merge(dim_df, on='CONDITION')

            # Rename 'CONDITION' column to 'Condition'
            merged_df.rename(columns={'CONDITION': 'Condition'}, inplace=True)

            # Add Patient_ID column
            merged_df.insert(0, 'Patient_ID', patient_id)

            # Merge with protocol data to add Start_Time and End_Time
            patient_protocols = protocol_data[protocol_data['Patient_ID'] == patient_id]
            merged_df = merged_df.merge(patient_protocols[['Condition', 'Start_Time', 'End_Time']],
                                        on='Condition', how='left')

            # Reorder columns: Move 'Condition', 'Start_Time', and 'End_Time' after 'Patient_ID'
            cols = merged_df.columns.tolist()
            for col in ['Condition', 'Start_Time', 'End_Time']:
                cols.insert(1, cols.pop(cols.index(col)))
            merged_df = merged_df[cols]

            return merged_df
        else:
            print(f"Missing one or more survey files for {patient_id}. Skipping merge.")
            return None

    except Exception as e:
        print(f"Error merging survey data for {patient_id}: {e}")
        return None

# Process all patient survey data
for file in os.listdir(output_folder_quest):
    if file.endswith("_PANAS.csv"):  # Identify patients by PANAS file existence
        patient_id = file.split("_")[0]  # Extract patient ID
        merged_survey_data = merge_surveys(patient_id, output_folder_quest, protocol_data)
        if merged_survey_data is not None:
            all_survey_data.append(merged_survey_data)

# Compile all patients' survey data into a single DataFrame and save to CSV
if all_survey_data:
    df_all_survey_data = pd.concat(all_survey_data, ignore_index=True)
    df_all_survey_data.to_csv(output_merged_file, index=False)


### Label compiled questionnaire data

In [ ]:
# Load the compiled survey data
survey_file = "/content/drive/MyDrive/Capstone/compiled_survey_data.csv"
df_survey = pd.read_csv(survey_file)

# Define PANAS question groups
orig_panas_pa = ["PANAS_Active", "PANAS_Interested", "PANAS_Inspired", "PANAS_Strong",
                 "PANAS_Excited", "PANAS_Proud", "PANAS_Enthusiastic", "PANAS_Alert",
                 "PANAS_Determined", "PANAS_Attentive"]

orig_panas_na = ["PANAS_Distressed", "PANAS_Annoyed", "PANAS_Strong", "PANAS_Guilty",
                 "PANAS_Scared", "PANAS_Hostile", "PANAS_Irritable", "PANAS_Ashamed",
                 "PANAS_Nervous", "PANAS_Jittery", "PANAS_Afraid"]

mod_panas_pa = orig_panas_pa + ["PANAS_Happy"]
mod_panas_na = orig_panas_na + ["PANAS_Stressed", "PANAS_Frustrated", "PANAS_Angry", "PANAS_Irritated", "PANAS_Sad"]

# Define STAI question groups (Reversed and Normal)
stai_normal = ["STAI_nervous", "STAI_jittery", "STAI_worried"]
stai_reversed = ["STAI_ease", "STAI_relaxed", "STAI_pleasant"]

# Reverse STAI scores (1↔4, 2↔3)
reverse_mapping = {1: 4, 2: 3, 3: 2, 4: 1}
for col in stai_reversed:
    df_survey[col] = df_survey[col].map(reverse_mapping)

# Compute PANAS Scores
df_survey["Orig_PANAS_PA"] = df_survey[orig_panas_pa].sum(axis=1)
df_survey["Orig_PANAS_NA"] = df_survey[orig_panas_na].sum(axis=1)
df_survey["Mod_PANAS_PA"] = df_survey[mod_panas_pa].sum(axis=1)
df_survey["Mod_PANAS_NA"] = df_survey[mod_panas_na].sum(axis=1)

# Self-Reported Stress Label
df_survey["Stress_label"] = df_survey["PANAS_Stressed"] - 1  # Shift values to match (0 = no stress, 1 = slight stress, etc.)

# Compute Modified STAI Score
df_survey["Mod_STAI"] = df_survey[stai_normal + stai_reversed].sum(axis=1)

# SAM Questionnaire Labels
df_survey["Valence_label"] = df_survey["SAM_valence"]
df_survey["Arousal_label"] = df_survey["SAM_arousal"]

# Save the updated dataset
output_labeled_file = "/content/drive/MyDrive/Capstone/labeled_survey_data.csv"
df_survey.to_csv(output_labeled_file, index=False)


### Update questionnaire labels

In [ ]:
# Load the compiled survey data
survey_file = "/content/drive/MyDrive/Capstone/compiled_survey_data.csv"
df_survey = pd.read_csv(survey_file)

# Define PANAS question groups
orig_panas_pa = ["PANAS_Active", "PANAS_Interested", "PANAS_Inspired", "PANAS_Strong",
                 "PANAS_Excited", "PANAS_Proud", "PANAS_Enthusiastic", "PANAS_Alert",
                 "PANAS_Determined", "PANAS_Attentive"]

orig_panas_na = ["PANAS_Distressed", "PANAS_Annoyed", "PANAS_Strong", "PANAS_Guilty",
                 "PANAS_Scared", "PANAS_Hostile", "PANAS_Irritable", "PANAS_Ashamed",
                 "PANAS_Nervous", "PANAS_Jittery", "PANAS_Afraid"]

# Define STAI question groups (Reversed and Normal)
stai_normal = ["STAI_nervous", "STAI_jittery", "STAI_worried"]
stai_reversed = ["STAI_ease", "STAI_relaxed", "STAI_pleasant"]

# Reverse STAI scores (1↔4, 2↔3)
reverse_mapping = {1: 4, 2: 3, 3: 2, 4: 1}
for col in stai_reversed:
    df_survey[col] = df_survey[col].map(reverse_mapping)

# Compute PANAS Scores (Only original PANAS PA and NA)
df_survey["PANAS_PA"] = df_survey[orig_panas_pa].sum(axis=1)
df_survey["PANAS_NA"] = df_survey[orig_panas_na].sum(axis=1)

# Map Stress label: 1 → 0 (no stress), 2 & 3 → 1 (low stress), 4 & 5 → 2 (high stress)
df_survey["Stress_label"] = df_survey["PANAS_Stressed"].map({1: 0, 2: 1, 3: 1, 4: 2, 5: 2})

# Compute STAI Score
df_survey["Mod_STAI"] = df_survey[stai_normal + stai_reversed].sum(axis=1)

# Map Anxiety Level: 0 (no to low anxiety) = STAI 4-9, 1 (moderate) = STAI 10-12, 2 (High) = STAI 13-24
df_survey["Anxiety_Level"] = df_survey["Mod_STAI"].apply(lambda x: 0 if x <= 9 else (1 if x <= 12 else 2))

# Depression Label: 1 (depressed) if valence < 5 and arousal < 5, else 0
df_survey["Depression_label"] = ((df_survey["SAM_valence"] < 5) & (df_survey["SAM_arousal"] < 5)).astype(int)

# Save the updated dataset
output_labeled_file = "/content/drive/MyDrive/Capstone/updated_labeled_survey_data.csv"
df_survey.to_csv(output_labeled_file, index=False)


## Extracting EDA and HRV features in 60s (1 min) intervals for all patients

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
import neurokit2 as nk
from scipy.stats import linregress

# Paths
data_folder = '/content/drive/MyDrive/Capstone/WESAD'
protocol_file = '/content/drive/MyDrive/Capstone/output_quest_data/compiled_protocol_data.csv'
output_file = '/content/drive/MyDrive/Capstone/eda_hrv_features_across_conditions_60s.csv'

# Load protocol times data
protocol_data = pd.read_csv(protocol_file)

# Define time split window size (in seconds)
split_window_size = 60
fs = 700  # Sampling rate (Hz)

# Initialize list to collect all features
all_features = []

# Function to process a single patient with ordered time-split windows
def process_patient_time_splits(patient_id, data_folder, protocol_data):
    pkl_path = os.path.join(data_folder, patient_id, f'{patient_id}.pkl')

    # Load patient data
    with open(pkl_path, 'rb') as f:
        patient_data = pickle.load(f, encoding='latin1')

    # Extract chest EDA and ECG signals
    chest_EDA = patient_data['signal']['chest']['EDA'].flatten()
    chest_ECG = patient_data['signal']['chest']['ECG'].flatten()
    total_duration = len(chest_EDA) / fs  # Convert samples to seconds

    # Filter protocol data for the specific patient
    patient_protocols = protocol_data[protocol_data['Patient_ID'] == patient_id]

    # Create a timeline for segment labeling
    condition_labels = np.full(int(total_duration // split_window_size) + 1, "Transition", dtype=object)

    # Assign condition labels based on protocol start and end times
    for _, row in patient_protocols.iterrows():
        condition = row['Condition']
        start_time = row['Start_Time']
        end_time = row['End_Time']

        if pd.isnull(start_time) or pd.isnull(end_time):
            continue

        start_idx = int(start_time // split_window_size)
        end_idx = int(end_time // split_window_size)

        condition_labels[start_idx:end_idx + 1] = condition  # Assign condition label

    patient_features = []

    # Process time-split windows
    num_samples_per_split = split_window_size * fs
    num_splits = len(chest_EDA) // num_samples_per_split

    for split_idx in range(num_splits):
        start_split = split_idx * num_samples_per_split
        end_split = start_split + num_samples_per_split

        eda_segment = chest_EDA[start_split:end_split]
        ecg_segment = chest_ECG[start_split:end_split]

        if len(eda_segment) < num_samples_per_split or len(ecg_segment) < num_samples_per_split:
            continue  # Skip incomplete segments

        # Determine segment condition label
        segment_condition = condition_labels[split_idx]

        # Process EDA signal
        eda_signals, eda_info = nk.eda_process(eda_segment, sampling_rate=fs)

        # Extract EDA components
        tonic_eda = eda_signals['EDA_Tonic']
        scr_peaks = eda_signals['SCR_Peaks']

        # Compute statistics for EDA components
        eda_features = {
            'SCL_Mean': np.mean(tonic_eda),  # Skin Conductance Level (SCL)
            'SCL_STD': np.std(tonic_eda),
            'SCR_Peaks_Count': len(scr_peaks.dropna()),  # Number of SCR peaks
            'EDA_Raw_Mean': np.mean(eda_segment),
            'EDA_Raw_STD': np.std(eda_segment),
            'EDA_Raw_Dynamic_Range': np.max(eda_segment) - np.min(eda_segment)
        }

        # Process ECG signal
        ecg_cleaned = nk.ecg_clean(ecg_segment, sampling_rate=fs)
        peaks, info = nk.ecg_peaks(ecg_cleaned, sampling_rate=fs, correct_artifacts=True)

        # Extract ECG statistics
        ecg_features = {
            'ECG_Mean': np.mean(ecg_segment),
            'ECG_Min': np.min(ecg_segment),
            'ECG_Max': np.max(ecg_segment),
            'ECG_Median': np.median(ecg_segment),
            'ECG_STD': np.std(ecg_segment)
        }

        # Extract HRV features if enough R-peaks are detected
        if len(peaks['ECG_R_Peaks']) >= 2:
            hrv_metrics = nk.hrv(peaks, sampling_rate=fs, show=False)

            # Extract required HRV features
            hrv_features = {
                'HRV_RMSSD': hrv_metrics['HRV_RMSSD'].iloc[0] if 'HRV_RMSSD' in hrv_metrics else np.nan,
                'HRV_SDNN': hrv_metrics['HRV_SDNN'].iloc[0] if 'HRV_SDNN' in hrv_metrics else np.nan,
                'HRV_pNN50': hrv_metrics['HRV_pNN50'].iloc[0] if 'HRV_pNN50' in hrv_metrics else np.nan,
                'HRV_SDANN': hrv_metrics['HRV_SDANN'].iloc[0] if 'HRV_SDANN' in hrv_metrics else np.nan,
                'HRV_MeanRR': hrv_metrics['HRV_MeanNN'].iloc[0] if 'HRV_MeanNN' in hrv_metrics else np.nan,
                'HRV_MedianRR': hrv_metrics['HRV_MedianNN'].iloc[0] if 'HRV_MedianNN' in hrv_metrics else np.nan,
                'HRV_HF': hrv_metrics['HRV_HF'].iloc[0] if 'HRV_HF' in hrv_metrics else np.nan,
                'HRV_LF': hrv_metrics['HRV_LF'].iloc[0] if 'HRV_LF' in hrv_metrics else np.nan,
                'HRV_LF_HF_Ratio': hrv_metrics['HRV_LFHF'].iloc[0] if 'HRV_LFHF' in hrv_metrics else np.nan,
                'HRV_SampEN': hrv_metrics['HRV_SampEn'].iloc[0] if 'HRV_SampEn' in hrv_metrics else np.nan
            }
        else:
            hrv_features = {key: np.nan for key in [
                'HRV_RMSSD', 'HRV_SDNN', 'HRV_pNN50', 'HRV_SDANN',
                'HRV_MeanRR', 'HRV_MedianRR', 'HRV_HF', 'HRV_LF',
                'HRV_LF_HF_Ratio', 'HRV_SampEN'
            ]}

        # Append extracted features
        patient_features.append({
            'Patient_ID': patient_id,
            'Condition': segment_condition,  # Assigned condition label (or "Transition")
            'Time_Split': split_idx + 1,
            **eda_features,
            **ecg_features,
            **hrv_features
        })

    return patient_features

# Loop through all patient folders and process
for patient_folder in os.listdir(data_folder):
    patient_id = patient_folder  # Patient ID corresponds to folder name (e.g., 'S2')
    patient_path = os.path.join(data_folder, patient_folder)

    if os.path.isdir(patient_path) and os.path.exists(os.path.join(patient_path, f'{patient_id}.pkl')):
        patient_features = process_patient_time_splits(patient_id, data_folder, protocol_data)
        all_features.extend(patient_features)

# Convert all features to a DataFrame and save to CSV
df_all_features = pd.DataFrame(all_features)
df_all_features.to_csv(output_file, index=False)

## Labeling Extracted Features

In [ ]:
patient_features_60s = pd.read_csv("/content/drive/MyDrive/Capstone/eda_hrv_features_across_conditions_60s.csv")
df_labels = pd.read_csv("/content/drive/MyDrive/Capstone/updated_labeled_survey_data.csv")
df_labeled_patient_features_60s = patient_features_60s.merge(df_labels, on=["Patient_ID", "Condition"], how="left")
df_labeled_patient_features_60s.to_csv("/content/drive/MyDrive/Capstone/labeled_patient_eda_hrv_features_60s.csv", index=False)